# 07 Segmentación de Clientes con Modelo RFM

En este ejercicio vamos a transformar datos transaccionales crudos en segmentos de clientes accionables.

**Objetivo:** Transformar el registro histórico de transacciones (datos crudos) en una Matriz de Segmentación RFM estratégica para responder:

* ¿Quiénes son nuestros clientes "Campeones"? (Compran mucho y seguido).
* ¿Quiénes son nuestros clientes "En Riesgo"? (Compraban mucho antes, pero se han ido).
* ¿Quiénes son "Leales"? (Compran seguido).

**Pasos:**
1. Generación, Carga y Exploración de Datos
2. Cálculo de Métricas RFM
3. Scoring
4. Segmentación Estratégica para la Toma de Decisiones
5. Visualización

## Parte 1: Generación, Carga y Exploración de Datos

In [1]:
import pandas as pd
import numpy as np
import datetime

np.random.seed(42)
n_transacciones = 10000
n_clientes = 500

# Generación de datos aleatorios
fechas = pd.date_range(start='2025-01-01', end='2026-01-09', periods=n_transacciones)
fechas = np.random.choice(fechas, n_transacciones) # Desordenar fechas
cliente_ids = np.random.randint(1000, 1000 + n_clientes, n_transacciones)
cantidades = np.random.randint(1, 10, n_transacciones)
precios = np.round(np.random.uniform(10, 100, n_transacciones), 2)

# Crear DataFrame
df = pd.DataFrame({
    'ID_Transaccion': [f'TRX-{i}' for i in range(n_transacciones)],
    'ID_Cliente': cliente_ids,
    'Fecha': fechas,
    'Cantidad': cantidades,
    'Precio_Unitario': precios
})

# Calcular Total y Guardar
df['Total'] = df['Cantidad'] * df['Precio_Unitario']

In [2]:
df

,ID_Transaccion,ID_Cliente,Fecha,Cantidad,Precio_Unitario,Total
0,TRX-0,1472,2025-09-29 04:45:17.551755176,8,30.47,243.76
1,TRX-1,1094,2025-02-02 01:56:56.381638163,3,46.83,140.49
2,TRX-2,1368,2025-07-21 01:36:38.019801980,7,23.94,167.58
3,TRX-3,1296,2025-07-13 15:26:52.601260126,2,58.22,116.44
4,TRX-4,1080,2025-08-02 21:35:24.572457244,4,37.69,150.76
...,...,...,...,...,...,...
9995,TRX-9995,1465,2025-06-24 11:16:08.856885688,7,61.03,427.21
9996,TRX-9996,1215,2025-03-24 17:45:07.830783078,9,47.48,427.32
9997,TRX-9997,1031,2025-02-21 19:33:25.760576057,5,57.23,286.15
9998,TRX-9998,1456,2025-06-09 12:15:28.892889288,6,95.73,574.38


## Parte 2: Cálculo de Métricas RFM

Debes crear un nuevo DataFrame donde cada fila sea un Cliente único (ID_Cliente) y las columnas sean sus métricas.

Calcula las 3 métricas:
* Recencia (R): Días transcurridos entre hoy y la última compra del cliente.
* Frecuencia (F): Número de facturas únicas por cliente.
* Monetario (M): Suma total del dinero gastado.

In [3]:
hoy = pd.to_datetime('today')

rfm = pd.DataFrame(df.groupby('ID_Cliente').agg({
    'Fecha': lambda x: (hoy - x.max()).days
}))

# Renombrar columnas para claridad
rfm.rename(columns={
    'Fecha': 'Recencia'
}, inplace=True)
rfm

,Recencia
ID_Cliente,
1000,64
1001,36
1002,18
1003,24
1004,15
...,...
1495,48
1496,40
1497,15


In [4]:

hoy = df['Fecha'].max() + pd.Timedelta(days=1)

rfm = df.groupby('ID_Cliente').agg({
    'Fecha': lambda x: (hoy - x.max()).days,
    'ID_Transaccion': 'count',
    'Total': 'sum'
}).reset_index()

rfm.columns = ['ID_Cliente', 'Recencia', 'Frecuencia', 'Monetario']

rfm.head()


,ID_Cliente,Recencia,Frecuencia,Monetario
0,1000,58,18,4435.15
1,1001,30,14,3554.99
2,1002,12,18,5527.68
3,1003,18,19,4643.73
4,1004,9,16,5050.27


## Parte 3: Scoring

Usa la función pd.qcut para dividir los datos en terciles:

Para Frecuencia y Monetario:

* Etiqueta 3 = Valores más altos (Mejor).
* Etiqueta 2 = Valores medios.
* Etiqueta 1 = Valores más bajos (Peor).

Para Recencia la lógica es inversa: Un valor bajo de recencia es mejor que un valor alto.

* Etiqueta 3 = Valores más bajos de días (Compró hace poco).
* Etiqueta 2 = Valores medios.
* Etiqueta 1 = Valores más altos de días (Hace mucho no compra).

In [5]:

rfm['R_Score'] = pd.qcut(rfm['Recencia'], q=3, labels=[3, 2, 1])


rfm['F_Score'] = pd.qcut(rfm['Frecuencia'], q=3, labels=[1, 2, 3])


rfm['M_Score'] = pd.qcut(rfm['Monetario'], q=3, labels=[1, 2, 3])


rfm[['R_Score', 'F_Score', 'M_Score']] = rfm[['R_Score', 'F_Score', 'M_Score']].astype(int)

rfm.head()


,ID_Cliente,Recencia,Frecuencia,Monetario,R_Score,F_Score,M_Score
0,1000,58,18,4435.15,1,1,1
1,1001,30,14,3554.99,1,1,1
2,1002,12,18,5527.68,2,1,2
3,1003,18,19,4643.73,2,2,1
4,1004,9,16,5050.27,3,1,2


## Parte 4: Segmentación Estratégica

Crea una columna llamada Segmento basada en las siguientes reglas lógicas:

| Segmento | Regla | Acción de Negocio |
| --- | ---- | --- |
| Campeones | R=3, F=3, M=3 | No molestar |
| Leales | F=3 (Cualquier R o M) | Intentar subir su ticket promedio (Up-selling) |
| En Riesgo | R=1, F=3 y/o M=3 | URGENTE: Enviar promoción agresiva para reactivarlos |
| Perdidos | R=1, F=1, M=1 | No invertir presupuesto de marketing.
| Nuevos | R=3, F=1 | Programa de bienvenida (Onboarding).

In [6]:
def segmentar_cliente(row):
    if row['R_Score'] == 3 and row['F_Score'] == 3 and row['M_Score'] == 3:
        return 'Campeones'
    elif row['R_Score'] == 1 and row['F_Score'] == 1 and row['M_Score'] == 1:
        return 'Perdidos'
    elif row['R_Score'] == 1 and (row['F_Score'] == 3 or row['M_Score'] == 3):
        return 'En Riesgo'
    elif row['R_Score'] == 3 and row['F_Score'] == 1:
        return 'Nuevos'
    elif row['F_Score'] == 3:
        return 'Leales'
    else:
        return 'Otros'

rfm['Segmento'] = rfm.apply(segmentar_cliente, axis=1)

rfm.head()

,ID_Cliente,Recencia,Frecuencia,Monetario,R_Score,F_Score,M_Score,Segmento
0,1000,58,18,4435.15,1,1,1,Perdidos
1,1001,30,14,3554.99,1,1,1,Perdidos
2,1002,12,18,5527.68,2,1,2,Otros
3,1003,18,19,4643.73,2,2,1,Otros
4,1004,9,16,5050.27,3,1,2,Nuevos
